# 데이터 읽기 및 전처리

In [11]:
pip install -U sentence-transformers

  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=120999 sha256=841ed9c396604cf87a641a0bd9ca778802f270d12edda46f1f60073400c5907e
  Stored in directory: c:\users\noa\appdata\local\pip\cache\wheels\52\19\88\6625593382e23a926740e6fcee0f2df0a0de25766094842a28
Successfully built sentence-transformers
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install -e .

Note: you may need to restart the kernel to use updated packages.


ERROR: File "setup.py" not found. Directory cannot be installed in editable mode: C:\Users\NoA\NLU


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from IPython.core.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_rows', 999) # pd.options.display.max_rows = 999
pd.set_option('display.max_columns', 999) # pd.options.display.max_columns = 999
pd.set_option('display.width', 1000)

# 한글폰트 적용
plt.rcParams['font.family'] = 'Malgun Gothic'

In [2]:
absolute_file_address = input("")

# read text data set
text_set = pd.read_csv(absolute_file_address)
text_set = text_set.fillna("")
text_set

final_20211207121745.csv


,Unnamed: 0,url,title,text
0,0,https://blog.naver.com/1457dudgus/222565835669,"대전 궁동 맛집, 워터에이징 궁동고기집 충남대맛집 ""투더돼지""","\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n궁동맛집 ""투더돼지""\n\..."
1,1,https://blog.naver.com/jangingin/222544329875,"충남대맛집, 매콤한 국물떡볶이와 꼬마김밥으로 간단하게 :)","\n\n\n\n\n오늘 백신접종하고, 학교에서 저녁에 수업이 있는데집에 들렀다 가기..."
2,2,https://blog.naver.com/apfjtzzz/222354696565,충남대 맛집 :: 삼각김밥의 클라쓰를 바꾼 강다짐,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n​남편은 예전부터 편의점 ...
3,3,https://blog.naver.com/help1207/222548639407,[ 대전 충남대맛집 ] [ 부쳐샵 충남대점 ] 유성구 수제버거 전문점,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n부쳐샵 충남대점\n\n\n...
4,4,https://blog.naver.com/wd4367/222576151259,충남대 맛집 봉명동 양고기 진1926,\n\n\n\n\n안녕하세요어제 친구랑 만나서 양고기를 먹고왔어요주말에고기를 먹은지...
...,...,...,...,...
2810,2810,https://blog.naver.com/dbsdustn007/222537839304,"호동 만다린 [ 대전 중국집 , 호동 짜장면 맛집 ]",\n\n\n\n\n​​​​​​​​​​​​​​​\n\n\n\n \n\n\n\n\n\n...
2811,2811,https://blog.naver.com/hopeu_self90/222300219919,제주 성산 삼달리 '호동이식당' 로컬 맛집,\n\n\n\n\n\n\n호동이 식당\n제주특별자치도 서귀포시 성산읍 삼달리 80-...
2812,2812,https://blog.naver.com/sparkdanmuji/222282109032,"삼겹살, 목살이 더 맛있는 돼지갈비 맛집 춘천 후평동 호동숯불갈비 솔직 후기",\n\n\n\n\n도단이아빠가 오랜만에 친한 회사 동료들과 저녁식사를 먹었어요메뉴를...
2813,2813,https://blog.naver.com/ejkim0422/221992057540,[대전 호동 신상카페 추천]모암로7번길 커피•디저트 맛집,\n\n\n\n\n안녕하세요 :)뉴스킨 나나랜드 은쥬입니다.💕​오늘 모처럼 대전나들...


In [3]:
text_set.shape

(2815, 4)

In [4]:
text_set = text_set.astype({'text':'string', 'url':'string'})
text_set.dtypes

Unnamed: 0     int64
url           string
title         object
text          string
dtype: object

# 데이터 분석 실행

In [5]:
import copy
import openpyxl
import csv

import re
from sentence_transformers import SentenceTransformer, util

f = open('submission.csv', 'w', newline='')
wr = csv.writer(f)
# 과도한 정보 표기 , 고유 단어 사용, 고료 표기, 유사한 문장 사용, 유사한 문장 사용
wr.writerow(["url", "use_tmi_words_value", "similar_sentence_value", "commissional_words_value"])

# Sentence-Transformers 패키지의 RoBERTa 알고리즘 사용
model = SentenceTransformer('paraphrase-distilroberta-base-v1')

individual_text_set = list()
for text in text_set['text']:
    lines = text.split('\n')
    lines = [v for v in lines if v]
            
    individual_text_set = individual_text_set + [v for v in lines if len(v) > 5]
    
embeddings1 = model.encode(individual_text_set, convert_to_tensor=True)

In [6]:
analysis_count = 0
while analysis_count < len(text_set):
    
    ### 전처리 ###
    print("Analysis position:", analysis_count, "->", analysis_count+5)
    
    if analysis_count+5 < len(text_set)-1:
        text_sub_set = text_set[analysis_count : analysis_count+5]
    else:
        text_sub_set = text_set[analysis_count : len(text_set)-1]
        
    
    # matrix(DataFrame)에서 text데이터를 가져온다.
    text_list = text_sub_set['text']
    url_list = text_sub_set['url']
    url_list = url_list.tolist()
    
    
    
    # text 분할
    individual_text_sub_set = list()
    individual_text_sub_set_size = list()
    for text in text_list:
        lines = text.split('\n')
        lines = [v for v in lines if v]
        lines = [v for v in lines if len(v) > 5]
        
        if(len(lines) == 0):
            lines.append("NONE")
            
        individual_text_sub_set = individual_text_sub_set + lines
        individual_text_sub_set_size.append(len(lines))
        
        
    
    
    ### 무의미한 문장 찾기 ###
    
    use_tmi_words_value = np.zeros(shape=(len(text_list),), dtype=np.int64)

    # 주소
    address_keywords = ["인천", "서울", "경기", "강원", "충청", "충남", "세종", "충북", "대전", "경상", "경북", "경남", "대구", "전라", "전남", "전북", "울산", "부산"]
    # 전화번호
    phoneNumRegex = re.compile(r'((\d{2}|\(\d{2}\)|\d{3}|\(\d{3}\))?(|-|\.)?(\d{3}|\d{4})(\s|-|\.)(\d{4}))')
    # 불규칙 키워드 (피드백 조정)
    irregular_keywords = ['수상', '대회', '출연', '전화번호', '전화', '번호', '운영시간', '운영 시간', '영업시간', '영업 시간']



    dest_keywords = address_keywords + irregular_keywords
    for i in range(0, len(text_list)):
        for line in text_list.tolist()[i].split('\n'):
            if(line == "\n"):
                continue

            for keyword in dest_keywords:
                if(line.find(keyword) != -1):
                    use_tmi_words_value[i] = use_tmi_words_value[i] + 1

            use_tmi_words_value[i] = use_tmi_words_value[i] + len(phoneNumRegex.findall(line))
            
    print("use_tmi_words_value:", use_tmi_words_value)
    
    
    
    
    ### 문장 유사도 분석 ###
    
    similar_sentence_value = np.zeros(shape=(len(text_list),), dtype=np.int64)        

    embeddings2 = model.encode(individual_text_sub_set, convert_to_tensor=True)
    cosine_scores = util.pytorch_cos_sim(embeddings2, embeddings1) # 코사인 유사도 공식
    
    
    for index in range(0, len(individual_text_sub_set)):
        if(individual_text_sub_set[index] == "NONE"):
            continue
        
        temp = cosine_scores[index]
        
        for i in temp.argsort(descending=True):
            if(cosine_scores[index][i] < 0.95):
                break
                
                
            
            temp_pos = individual_text_sub_set_size[0]
            for l in range(0, len(individual_text_sub_set_size)):
                if(index < temp_pos):
                    similar_sentence_value[l] = similar_sentence_value[l] + 1
                    break
                   
                else:
                   if(l+1 < len(individual_text_sub_set_size)):
                       temp_pos = temp_pos + individual_text_sub_set_size[l+1]
                    
    print("similar_sentence_value", similar_sentence_value)
    
    
    
    
    ### 공정위 문구 분석 ###
    
    commissional_words = ["협찬", "고료", "광고", "후원", "원고", "지원", "제공", "업체", "서비스"]

    commissional_words_value = np.zeros(shape=(len(text_list),), dtype=np.int64)

    for index in range(0, len(text_list)):
        for word in commissional_words:
            if(len(text_list) == 0):
                continue
            if(text_list.tolist()[index].find(word) != -1):
                commissional_words_value[index] = 1

    print("commissional_words_value:", commissional_words_value)
    
    for row in range(0, 5):
        if(len(use_tmi_words_value) <= row):
            break
        wr.writerow([url_list[row], use_tmi_words_value[row], similar_sentence_value[row], commissional_words_value[row]])
        
        
    analysis_count = analysis_count+5
    print("\nData cast")
    
f.close()

print("Analysis finish")

Analysis position: 0 -> 5
use_tmi_words_value: [ 8  9 12 15  5]
similar_sentence_value [ 573  914  706 1535 1772]
commissional_words_value: [1 1 0 1 0]

Data cast
Analysis position: 5 -> 10
use_tmi_words_value: [ 9  3 23  6  8]
similar_sentence_value [1720   44 1691  213 1353]
commissional_words_value: [0 0 0 0 1]

Data cast
Analysis position: 10 -> 15
use_tmi_words_value: [14 13 12 10 11]
similar_sentence_value [1567  575  402  501  116]
commissional_words_value: [1 0 0 1 0]

Data cast
Analysis position: 15 -> 20
use_tmi_words_value: [ 7 11  5  8  9]
similar_sentence_value [ 102 1317  185 2920  126]
commissional_words_value: [1 0 0 0 0]

Data cast
Analysis position: 20 -> 25
use_tmi_words_value: [3 0 3 7 4]
similar_sentence_value [ 264   40  185 1113  155]
commissional_words_value: [0 0 0 0 0]

Data cast
Analysis position: 25 -> 30
use_tmi_words_value: [ 2 11  2  2  6]
similar_sentence_value [ 90 439  40 125 137]
commissional_words_value: [0 0 0 0 0]

Data cast
Analysis position: 30 -

similar_sentence_value [1079  684  410   96  367]
commissional_words_value: [1 0 0 0 0]

Data cast
Analysis position: 255 -> 260
use_tmi_words_value: [61  6  5  4  8]
similar_sentence_value [1001 1302  905  785  341]
commissional_words_value: [1 1 0 1 1]

Data cast
Analysis position: 260 -> 265
use_tmi_words_value: [ 1  3  2 20  8]
similar_sentence_value [  58 1309  472  314 1712]
commissional_words_value: [0 0 1 0 1]

Data cast
Analysis position: 265 -> 270
use_tmi_words_value: [ 5 37  6  9 30]
similar_sentence_value [ 241  888 2602 1184 4318]
commissional_words_value: [0 1 1 0 1]

Data cast
Analysis position: 270 -> 275
use_tmi_words_value: [35  5  4  8 12]
similar_sentence_value [1918  295   89 1550  867]
commissional_words_value: [0 0 0 0 1]

Data cast
Analysis position: 275 -> 280
use_tmi_words_value: [ 6 10  8  7  3]
similar_sentence_value [ 367 1026  506  453  184]
commissional_words_value: [0 1 0 1 0]

Data cast
Analysis position: 280 -> 285
use_tmi_words_value: [1 4 3 1 3]
sim

similar_sentence_value [205 360 244 284 129]
commissional_words_value: [0 0 1 0 1]

Data cast
Analysis position: 510 -> 515
use_tmi_words_value: [ 3  1  4 13  9]
similar_sentence_value [ 777   21 1430  300  297]
commissional_words_value: [1 0 0 0 0]

Data cast
Analysis position: 515 -> 520
use_tmi_words_value: [6 2 4 2 2]
similar_sentence_value [ 51  85  51 116 152]
commissional_words_value: [1 1 0 0 0]

Data cast
Analysis position: 520 -> 525
use_tmi_words_value: [3 1 8 4 7]
similar_sentence_value [ 16  42 582 396  41]
commissional_words_value: [0 0 0 0 0]

Data cast
Analysis position: 525 -> 530
use_tmi_words_value: [ 4  1  7  6 16]
similar_sentence_value [1297  676   32  576  508]
commissional_words_value: [0 0 0 0 0]

Data cast
Analysis position: 530 -> 535
use_tmi_words_value: [5 0 1 1 4]
similar_sentence_value [ 137   13  173   18 1630]
commissional_words_value: [0 0 0 0 1]

Data cast
Analysis position: 535 -> 540
use_tmi_words_value: [ 3  6  7 10  1]
similar_sentence_value [ 77 

similar_sentence_value [ 507 1166   53  104  201]
commissional_words_value: [0 1 0 0 0]

Data cast
Analysis position: 760 -> 765
use_tmi_words_value: [5 1 7 0 6]
similar_sentence_value [361 167 201  15 755]
commissional_words_value: [0 0 0 0 0]

Data cast
Analysis position: 765 -> 770
use_tmi_words_value: [ 3  6  6  4 10]
similar_sentence_value [1232  519  839  707   61]
commissional_words_value: [0 1 0 0 1]

Data cast
Analysis position: 770 -> 775
use_tmi_words_value: [5 7 2 5 3]
similar_sentence_value [413 208  84 310  25]
commissional_words_value: [0 0 0 0 0]

Data cast
Analysis position: 775 -> 780
use_tmi_words_value: [11  4  5  0  6]
similar_sentence_value [  74 1409   48  327  799]
commissional_words_value: [0 0 0 0 1]

Data cast
Analysis position: 780 -> 785
use_tmi_words_value: [19  4  2 10  7]
similar_sentence_value [ 693 1076  333  169  186]
commissional_words_value: [1 0 0 1 0]

Data cast
Analysis position: 785 -> 790
use_tmi_words_value: [ 3  8 13  8  9]
similar_sentence_v

similar_sentence_value [1444  498  204 1766  166]
commissional_words_value: [0 0 1 1 0]

Data cast
Analysis position: 1010 -> 1015
use_tmi_words_value: [4 8 3 5 2]
similar_sentence_value [251 608 397 957  45]
commissional_words_value: [0 0 0 0 0]

Data cast
Analysis position: 1015 -> 1020
use_tmi_words_value: [6 5 6 2 3]
similar_sentence_value [267 343  90 347  65]
commissional_words_value: [0 1 0 0 0]

Data cast
Analysis position: 1020 -> 1025
use_tmi_words_value: [ 5  3 13  3  6]
similar_sentence_value [ 462  308  372   64 1003]
commissional_words_value: [1 0 0 0 1]

Data cast
Analysis position: 1025 -> 1030
use_tmi_words_value: [16  1 11  4  5]
similar_sentence_value [ 121  248  779  644 1288]
commissional_words_value: [0 0 0 0 0]

Data cast
Analysis position: 1030 -> 1035
use_tmi_words_value: [11  5  2 10  9]
similar_sentence_value [435  19  30 147  41]
commissional_words_value: [1 0 0 0 0]

Data cast
Analysis position: 1035 -> 1040
use_tmi_words_value: [3 3 6 6 3]
similar_sentence

...

similar_sentence_value [297 809 156 245 462]
commissional_words_value: [0 1 0 1 1]

Data cast
Analysis position: 1760 -> 1765
use_tmi_words_value: [ 1 12  5  1  0]
similar_sentence_value [ 574 1117  461   31  788]
commissional_words_value: [0 0 0 1 0]

Data cast
Analysis position: 1765 -> 1770
use_tmi_words_value: [5 4 7 1 1]
similar_sentence_value [1576  532  159  411  443]
commissional_words_value: [0 0 0 0 1]

Data cast
Analysis position: 1770 -> 1775
use_tmi_words_value: [3 1 1 1 1]
similar_sentence_value [  92  215 2044  167  212]
commissional_words_value: [0 0 1 0 0]

Data cast
Analysis position: 1775 -> 1780
use_tmi_words_value: [4 3 2 3 1]
similar_sentence_value [ 296 2948  402   65  559]
commissional_words_value: [1 0 1 0 1]

Data cast
Analysis position: 1780 -> 1785
use_tmi_words_value: [1 0 3 1 5]
similar_sentence_value [  6   8 738 848 327]
commissional_words_value: [0 0 0 0 0]

Data cast
Analysis position: 1785 -> 1790
use_tmi_words_value: [ 9  7 17  3 12]
similar_sentence

similar_sentence_value [  51 1551   24  309 1469]
commissional_words_value: [0 1 1 1 0]

Data cast
Analysis position: 2010 -> 2015
use_tmi_words_value: [ 6  5  4 14  1]
similar_sentence_value [1671  376  497  379  110]
commissional_words_value: [1 1 1 0 0]

Data cast
Analysis position: 2015 -> 2020
use_tmi_words_value: [5 5 7 4 8]
similar_sentence_value [ 17 161  79 807 369]
commissional_words_value: [0 1 0 0 0]

Data cast
Analysis position: 2020 -> 2025
use_tmi_words_value: [10  8  5  8  7]
similar_sentence_value [224 942 353 979 694]
commissional_words_value: [1 0 1 1 1]

Data cast
Analysis position: 2025 -> 2030
use_tmi_words_value: [ 5  6 12  6  1]
similar_sentence_value [314 205 256 550 207]
commissional_words_value: [0 1 0 1 0]

Data cast
Analysis position: 2030 -> 2035
use_tmi_words_value: [4 8 8 3 1]
similar_sentence_value [ 42 277  88 328 190]
commissional_words_value: [0 1 1 0 0]

Data cast
Analysis position: 2035 -> 2040
use_tmi_words_value: [10  1  8 18 14]
similar_sentence

similar_sentence_value [1186  140  554  321  284]
commissional_words_value: [0 0 0 0 0]

Data cast
Analysis position: 2260 -> 2265
use_tmi_words_value: [3 7 4 1 0]
similar_sentence_value [1213  151  994   89   95]
commissional_words_value: [0 0 0 0 0]

Data cast
Analysis position: 2265 -> 2270
use_tmi_words_value: [5 6 5 7 1]
similar_sentence_value [578 119 306 314 233]
commissional_words_value: [1 0 1 0 1]

Data cast
Analysis position: 2270 -> 2275
use_tmi_words_value: [14  2  4  4  5]
similar_sentence_value [4826  202 1486  163 1792]
commissional_words_value: [1 0 0 0 0]

Data cast
Analysis position: 2275 -> 2280
use_tmi_words_value: [3 6 8 6 7]
similar_sentence_value [ 135  359  228 1382  427]
commissional_words_value: [1 0 1 1 1]

Data cast
Analysis position: 2280 -> 2285
use_tmi_words_value: [6 0 9 3 1]
similar_sentence_value [ 189   71   86 2718  692]
commissional_words_value: [1 1 1 1 1]

Data cast
Analysis position: 2285 -> 2290
use_tmi_words_value: [2 7 5 2 8]
similar_sentence

similar_sentence_value [  52  251  173 1350 1328]
commissional_words_value: [0 1 0 0 1]

Data cast
Analysis position: 2510 -> 2515
use_tmi_words_value: [0 4 7 2 1]
similar_sentence_value [741  18  86   9  13]
commissional_words_value: [0 0 0 1 0]

Data cast
Analysis position: 2515 -> 2520
use_tmi_words_value: [31 11  2  7  1]
similar_sentence_value [2726  482  166   56  766]
commissional_words_value: [1 0 0 0 0]

Data cast
Analysis position: 2520 -> 2525
use_tmi_words_value: [2 2 5 7 6]
similar_sentence_value [1582  109  678  626 1127]
commissional_words_value: [1 0 0 0 1]

Data cast
Analysis position: 2525 -> 2530
use_tmi_words_value: [ 5  3 13 12  1]
similar_sentence_value [1243 1364 1863  933 1385]
commissional_words_value: [1 0 0 0 0]

Data cast
Analysis position: 2530 -> 2535
use_tmi_words_value: [12 12  4  5  5]
similar_sentence_value [ 113  437  406 1461  932]
commissional_words_value: [0 0 0 0 0]

Data cast
Analysis position: 2535 -> 2540
use_tmi_words_value: [ 8  4 11  6 16]
s

similar_sentence_value [ 73 634 950 496 128]
commissional_words_value: [0 1 1 0 0]

Data cast
Analysis position: 2760 -> 2765
use_tmi_words_value: [11  8  5  6  7]
similar_sentence_value [859 165 158 374 298]
commissional_words_value: [0 0 1 0 0]

Data cast
Analysis position: 2765 -> 2770
use_tmi_words_value: [33 22 39  9  7]
similar_sentence_value [2639 1291 3335  654  342]
commissional_words_value: [0 1 0 1 1]

Data cast
Analysis position: 2770 -> 2775
use_tmi_words_value: [10  5  8  4 19]
similar_sentence_value [ 403 1997  128  741 1458]
commissional_words_value: [1 0 0 0 1]

Data cast
Analysis position: 2775 -> 2780
use_tmi_words_value: [ 4  3 15 12  5]
similar_sentence_value [678 774 774 218 319]
commissional_words_value: [1 0 1 0 1]

Data cast
Analysis position: 2780 -> 2785
use_tmi_words_value: [ 8  7 48  6  2]
similar_sentence_value [459 699 769  95 659]
commissional_words_value: [1 1 0 0 0]

Data cast
Analysis position: 2785 -> 2790
use_tmi_words_value: [4 6 1 1 1]
similar_sen

# 모듈 종료

In [18]:
print("text_module_finish")

text_module_finish
